In [1]:
from tkinter import *
from tkinter import Toplevel
from tkinter.messagebox import askyesno
from tkinter.filedialog import asksaveasfile
import logging


LOG_FORMAT = "%(asctime)-15s [%(oname)s] %(message)s"


class ScrolledText(Frame):

    def __init__(self, parent=None):
        Frame.__init__(self, parent)
        self.parent = parent
        self.text = self._make_scrolling_text()
        self.pack(expand=YES, fill=BOTH)

        self.menu = Menu(self, tearoff=0)
        self.menu.add_command(label="Copy", command=self.copy_text)
        self.parent.bind_class("Text", "<Button-2>", self.show_menu)

    def _make_scrolling_text(self):
        sbar = Scrollbar(self)
        text = Text(self, relief=SUNKEN)
        # Cross-link and move scrollbar to follow text
        sbar.config(command=text.yview)
        text.config(yscrollcommand=sbar.set)
        sbar.pack(side=RIGHT, fill=Y)
        text.pack(side=LEFT, expand=YES, fill=BOTH)
        return text

    def show_menu(self, event):
        self.menu.post(event.x_root, event.y_root)
        return self

    def copy_text(self):
        try:
            selection = self.text.selection_get()
            self.parent.clipboard_clear()
            self.parent.clipboard_append(str(selection))
        except TclError:
            return
        return self

    def set_text(self, text=''):
        self.set_text_widget_state("normal")
        self.text.insert(END, text)
        self.text.see(END)
        self.set_text_widget_state("disabled")
        return self

    def get_text(self):
        return self.text.get('1.0', END + '-1c')

    def save_text(self):
        fp = asksaveasfile()
        if fp is not None:
            fp.write(self.get_text())
            fp.close()
        return self

    def set_text_widget_state(self, state):
        self.text.config(state=state)
        return self

In [2]:
class WindowLoggingHandler(logging.Handler):
    def __init__(self, window=None, level=logging.NOTSET):
        logging.Handler.__init__(self, level)

        if window is None:
            self.window = Tk()
        else:
            self.window = window

        self.scrolling_text = ScrolledText(parent=self.window)
        self.scrolling_text.set_text_widget_state("disabled")
        self.scrolling_text.pack(side=TOP, expand=YES, fill=BOTH)

        save_btn = Button(
            master=self.window,
            text='Save As...',
            command=self.scrolling_text.save_text
        )
        save_btn.pack(side=RIGHT, anchor=S, padx=2, pady=2)

        close_btn = Button(
            master=self.window,
            text='Close',
            command=self.close_window
        )
        close_btn.pack(side=RIGHT, anchor=S, padx=2, pady=2)
        self.window.protocol("WM_DELETE_WINDOW", self.close_window)

    def close_window(self):
        if askyesno('Verify', 'Do you really want to quit?'):
            self.close()
            logger = logging.getLogger()
            logger.removeHandler(self)
            self.window.destroy()
            
    def hide_window(self):
        self.window.withdraw()
        
    def show_window(self):
        self.window.deiconify()
        self.window.lift()

    def emit(self, record):
        text = self.format(record) + '\n'
        self.scrolling_text.set_text(text)

    def mainloop(self):
        self.window.mainloop()

In [ ]:
if __name__ == '__main__':
    root = Tk()
    win = None
    log_window = None
    first_time = True
               
    def repeat():
        write_msg()
        root.after(1000, repeat)
    
    def write_msg():
        logging.warning(msg="Hello World!", extra={'oname': 1})
        return
    
    def doit():
        global log_window
        global win
        global root
        global first_time
        win = Toplevel(master=root)
        win.title('Enrich2 Log')
        logger = logging.getLogger()
        log_window = WindowLoggingHandler(window=win)
        formatter = logging.Formatter(LOG_FORMAT)
        log_window.setFormatter(formatter)
        if first_time:
            stream_handler = logging.StreamHandler()
            formatter = logging.Formatter(LOG_FORMAT)
            stream_handler.setFormatter(formatter)
            logging.basicConfig(
                format=LOG_FORMAT, handlers=[stream_handler, log_window]
            )
            first_time = False
        else:
            logging.getLogger().addHandler(log_window)
        log_window.mainloop()
    
    visible = True     
    def toggle():
        global visible
        if visible:
            btn.pack_forget()
            visible = False
        else:
            btn.pack()
            visible = True
        
    btn = Button(root, text="Make Terminal.", command=doit)
    btn2 = Button(root, text="Write to text.", command=write_msg)
    btn3 = Button(root, text="Starting writing", command=repeat)

    btn.pack(side=LEFT, expand=YES, fill=BOTH, padx=2, pady=2)
    btn2.pack(side=LEFT, expand=YES, fill=BOTH, padx=2, pady=2)
    btn3.pack(side=LEFT, expand=YES, fill=BOTH, padx=2, pady=2)

    root.mainloop()
